In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:

ohe = OneHotEncoder(sparse=False)

text = "raw data"
vocab = sorted(list(set(text)))
x_cat = np.array([vocab.index(c) for c in text])
x = ohe.fit_transform(x_cat.reshape(-1,1))

In [146]:
import tensorflow as tf

class Symbols:
    pass

def build_graph(time_steps=2, input_dims=6):
    graph = tf.Graph()
    syms = Symbols()  # just a container for variables we'll want to interact with later on

    with graph.as_default():
        
        # setup data
        syms.x = tf.placeholder(tf.float32, (None, time_steps, input_dims))
        
        # construct 2 layer lstm cell
        layer0 = tf.nn.rnn_cell.LSTMCell(num_units=4)
        layer1 = tf.nn.rnn_cell.LSTMCell(num_units=6)
        cell = tf.nn.rnn_cell.MultiRNNCell([layer0, layer1], state_is_tuple=True)
        syms.outputs, syms.last_state = tf.nn.dynamic_rnn(cell, syms.x, dtype=tf.float32)
        
        # get last rnn output
        last_index = syms.outputs.get_shape()[1] - 1
        syms.outputs_tmajor = tf.transpose(syms.outputs, [1,0,2])
        syms.last_output = tf.gather(syms.outputs_tmajor, last_index)
        
        # we use this mainly for test time
        syms.last_output_softmax = tf.nn.softmax(syms.last_output)
        
        # book keeping
        syms.saver = tf.train.Saver()
        
    return graph, syms

def do_train(graph, syms, data):
    total_len  = data.shape[0]
    input_dim  = data.shape[1]
    time_steps = 2
    batch_size = 3

    # setup additional operations for training
    with graph.as_default():
        # setup loss
        syms.y = tf.placeholder(tf.float32, (None, 6))
        syms.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(syms.last_output, syms.y))

        # training infrastructure
        syms.optimizer = tf.train.AdadeltaOptimizer(learning_rate=.1)
        syms.minimize = syms.optimizer.minimize(syms.loss)
        
    with tf.Session(graph=graph) as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(3):
            for t in range(total_len-time_steps-batch_size-2):
                batch_x = []
                batch_y = []
                for offset in range(batch_size):
                    start = t+offset
                    end = start+time_steps
                    x = data[start:end, :]
                    y = data[end+1]
                    batch_x.append(x)
                    batch_y.append(y)
            
                inputs = {}
                inputs[syms.x] = np.array(batch_x)
                inputs[syms.y] = np.array(batch_y)
                outputs = sess.run([syms.minimize, syms.last_output, syms.loss], inputs)
                print outputs[2]
                syms.saver.save(sess, "model_0.ckpt")

def do_eval():
    # with tf.Session(graph=graph) as sess:
    #     s.saver.restore(sess, "model_0.ckpt")
    #     inputs = {}
    #     inputs[s.x] = x.astype(np.float32)[:3, :].reshape(1, 3, 6)
    #     y2 = sess.run(s.outputs, inputs)
    pass
    


        
train_graph, syms = build_graph()
do_train(train_graph, syms, x)





1.80371
1.80366
1.80362


In [152]:
eval_graph, syms = build_graph(time_steps=2)
with tf.Session(graph=eval_graph) as sess:
    syms.saver.restore(sess, "model_0.ckpt")
    inputs = {}
    inputs[syms.x] = x.astype(np.float32)[:2, :].reshape(1, 2, 6)
    y_hat, state = sess.run([syms.last_output, syms.last_state], inputs)
    print y2
    print state[0].shape
    print state[1].shape


[[ 0.02649736 -0.00723264  0.00877121  0.01212392  0.00653472 -0.02457531]]
(1, 8)
(1, 12)
